In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import pandascharm as pc

from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62
from Bio.SubsMat.MatrixInfo import blosum45
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align import AlignInfo

from sklearn.preprocessing import scale 
from sklearn import model_selection

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso


from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate, GroupKFold
import scipy.stats as stats


C:\Users\memre\anaconda3\lib\site-packages\Bio\SubsMat\__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
def removeoutlier_col(df,cols):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df[[cols]] < (Q1 - 1.5 * IQR)) |(df[[cols]] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out

In [3]:
blosum62.update(((b,a),val) for (a,b),val in list(blosum62.items()))
blosum45.update(((b,a),val) for (a,b),val in list(blosum45.items()))

def score_pairwise(seq1, seq2, matrix, gap_s, gap_e, gap = True):
    for A,B in zip(seq1, seq2):
        diag = ('-'==A) or ('-'==B)
        yield (gap_e if gap else gap_s) if diag else matrix[(A,B)]
        gap = diag


In [4]:
#Sequence Representation Methods with Temperature

def encode_temp(encoding, output, df_clean, aln, esm1b , temperature):
    
    df_clean = df_clean.set_index('Index')
    
    ClustalAlign = AlignIO.read(aln, 'clustal')
    summary_align = AlignInfo.SummaryInfo(ClustalAlign )
    dframe = pc.from_bioalignment(ClustalAlign).transpose()
    sequences = dframe.loc[df_clean.index]
    
    y = df_clean['y_val']

    scaler = preprocessing.StandardScaler()
  
    
    if encoding == 'One-Hot-Encoder':

        one_hot = OneHotEncoder()
        encoded = one_hot.fit(sequences)
        sequences_encoded = encoded.transform(sequences).toarray()
        X = np.concatenate((sequences_encoded,df_clean[[temperature]]), axis =1)
        

        
           
    if encoding == 'Bag-of-Words':

        X = pd.DataFrame([ProteinAnalysis(i).count_amino_acids() for i in df_clean['Sequence']])
        X = np.concatenate((X,df_clean[[temperature]]), axis =1)

        
    if encoding == 'bigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                st = i+j
                all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-1):
                try:
                    check = temp[k] + temp[k+1]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.concatenate((X,df_clean[[temperature]]), axis =1)

    
    if encoding == 'trigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    st = i+j+k
                    all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-2):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.concatenate((X,df_clean[[temperature]]), axis =1)

        
        
    if encoding == 'quadrogram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    for l in lst:
                        st = i+j+k+l
                        all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-3):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]+temp[k+3]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        
        X = np.concatenate((X,df_clean[[temperature]]), axis =1)

    if encoding == 'BLOSUM62':

        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum62, -5, -1))
                j += 1
            i += 1

        X = np.concatenate((enc_seq,df_clean[[temperature]]), axis =1)

        
        
        
    if encoding == 'BLOSUM45':
        
        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum45, -5, -1))
                j += 1
            i += 1

        X = np.concatenate((enc_seq,df_clean[[temperature]]), axis =1)

        
    if encoding == 'ESM1b':
        

        encoded = esm1b.loc[df_clean.index]
        X = np.concatenate((encoded,df_clean[[temperature]]), axis =1)

    scaler.fit(X)
    X_scaled = scaler.transform(X)
    
    return X, y, X_scaled

In [5]:
def ml_process(encoding, output, df, aln, esm1b, temp=False):
    

    
    df['Relative Temperature'] = df['Reaction Temperature'] - df["Temperature Optimum"]

    df_clean = removeoutlier_col(df, 'Log'+output).copy()
    df_clean = df_clean.reset_index()

    # Create a mapping of unique sequences to unique codes
    sequence_to_code = {seq: f"ENZYME_{i+1}" for i, seq in enumerate(df_clean['Sequence'].unique())}

    # Map these codes to a new column in the DataFrame using .loc
    df_clean.loc[:, 'Sequence Code'] = df_clean['Sequence'].map(sequence_to_code)
    
    
    group_kfold = GroupKFold(n_splits=5)
    

    X, y, X_scaled = encode_temp(encoding, output, df_clean, aln, esm1b , 'Relative Temperature')
    
    y_scaled = y

    X_scaled, y_scaled, group_names_scaled = shuffle(X_scaled, y_scaled,df_clean['Sequence Code'], random_state=101)
    
    X, y , group_names = shuffle(X, y, df_clean['Sequence Code'], random_state=101)
    

    lm = LinearRegression()   
    scores_lm = cross_validate(lm, X_scaled, y_scaled, cv=group_kfold, groups=group_names_scaled,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_lm.update({"Algorithm": "Linear regression", 'Sequence Representation Method': encoding})
                               
    lasso_reg=Lasso()
    scores_lasso = cross_validate(lasso_reg,  X_scaled,y_scaled, cv=group_kfold,groups=group_names_scaled,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_lasso.update({"Algorithm": "LASSO", 'Sequence Representation Method': encoding})
                               
    rf = RandomForestRegressor(random_state=42)
    scores_rf = cross_validate(rf,  X, y, cv=group_kfold,groups=group_names,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_rf.update({"Algorithm": "Random Forest", 'Sequence Representation Method': encoding})    
    
    tree_reg=DecisionTreeRegressor()
    scores_tree_reg = cross_validate(tree_reg,  X, y, cv=group_kfold,groups= group_names,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_tree_reg.update({"Algorithm": "Decision Tree", 'Sequence Representation Method': encoding})    
    
    svr_reg = SVR()
    scores_svr_reg = cross_validate(svr_reg,  X_scaled, y_scaled, cv=group_kfold,groups=group_names_scaled,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_svr_reg.update({"Algorithm": "SVR", 'Sequence Representation Method': encoding})    
        
    
    mlp_reg = MLPRegressor(random_state=101, max_iter=100)
    scores_mlp_reg = cross_validate(mlp_reg,  X_scaled, y_scaled, cv=group_kfold,groups=group_names_scaled,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_mlp_reg.update({"Algorithm": "Neural Network", 'Sequence Representation Method': encoding})    
    
    en = ElasticNet()
    scores_en = cross_validate(en, X_scaled,y_scaled, cv=group_kfold,groups=group_names_scaled,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_tree_reg.update({"Algorithm": "Elastic Network", 'Sequence Representation Method': encoding})    
    
    xgb_reg = xgb.XGBRegressor()
    scores_xgb_reg = cross_validate(xgb_reg,  X, y, cv=group_kfold,groups=group_names,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_xgb_reg.update({"Algorithm": "XGBoost", 'Sequence Representation Method': encoding})    
    
    dfResults = pd.concat([pd.DataFrame(scores_lm), pd.DataFrame(scores_lasso), pd.DataFrame(scores_rf), pd.DataFrame(scores_tree_reg), pd.DataFrame(scores_svr_reg), pd.DataFrame(scores_mlp_reg), pd.DataFrame(scores_tree_reg), pd.DataFrame(scores_xgb_reg)])
        
    
    return dfResults


In [6]:
enzyme = 'betaGlucosidasewithMutants'

df = pd.read_excel('betaGlucosidasewithMutantsOptimumTemperature.xlsx')

output = 'pNP-Glc kcat/Km (1/smM)'

methods = ['One-Hot-Encoder', 'Bag-of-Words', 'bigram', 'trigram', 'quadrogram', 'BLOSUM45', 'BLOSUM62', 'ESM1b']

aln = enzyme +'.aln'

x = datetime.datetime.now()
date = str(x.year)+str(x.month)+str(x.day)

df['y_val']= df['Percentage Activity Depending on Optimum Temp']
df['Log'+output]= np.log10(df[output])




In [7]:
esm1b = pd.read_excel(enzyme+'ESM1b_embeddings.xlsx', index_col = 0)

In [8]:
summary=[]

for method in methods:
    dfR = ml_process(method, output , df, aln, esm1b, temp = False)
    print(dfR)
    summary.append(dfR)



     fit_time  score_time       test_r2   train_r2  \
0    0.846633    0.009291 -1.615277e+25  -0.366769   
1    0.869448    0.009348 -1.529483e+26  -1.444893   
2    0.783834    0.009470 -5.241066e+24  -0.519526   
3    0.758018    0.008214 -3.028922e+26  -5.915145   
4    0.712231    0.007190 -1.467878e+25  -2.199132   
0    0.191547    0.007893 -2.826904e-04   0.000000   
1    0.186089    0.009739 -9.258189e-03   0.000000   
2    0.195073    0.008739 -2.060961e-03   0.000000   
3    0.197555    0.009758 -4.953691e-03   0.000000   
4    0.191295    0.009272 -5.354791e-03   0.000000   
0  175.978457    0.041608  7.594621e-01   0.969333   
1  173.563160    0.033160  7.130374e-01   0.971119   
2  174.731113    0.038394  7.546227e-01   0.968419   
3  172.603507    0.035625  6.463598e-01   0.971181   
4  180.559243    0.031810  6.810106e-01   0.971955   
0    2.986325    0.008206  5.478437e-01   1.000000   
1    2.694448    0.010923  4.369551e-01   1.000000   
2    2.869674    0.010879  5

C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multi

   fit_time  score_time   test_r2  train_r2  test_neg_root_mean_squared_error  \
0  0.002995    0.003017  0.013787  0.251994                         -0.316198   
1  0.002007    0.002001  0.142479  0.220172                         -0.273693   
2  0.001216    0.002542  0.197105  0.206767                         -0.279388   
3  0.002000    0.002784  0.155257  0.212907                         -0.304207   
4  0.002998    0.002999  0.173264  0.211339                         -0.306304   
0  0.001001    0.003075 -0.000283  0.000000                         -0.318446   
1  0.002015    0.002001 -0.009258  0.000000                         -0.296922   
2  0.002704    0.002008 -0.002061  0.000000                         -0.312123   
3  0.002001    0.002233 -0.004954  0.000000                         -0.331802   
4  0.002002    0.002961 -0.005355  0.000000                         -0.337777   
0  2.377349    0.017934  0.752784  0.967181                         -0.158311   
1  2.320156    0.028683  0.6

    fit_time  score_time       test_r2  train_r2  \
0   0.044483    0.004094 -4.314799e+25  0.449992   
1   0.044852    0.003801 -1.498325e+24  0.414818   
2   0.040926    0.004008 -6.507499e+25  0.403578   
3   0.040390    0.002998 -2.744651e+23  0.408234   
4   0.044714    0.004000 -1.437291e+24  0.413971   
0   0.012254    0.006122 -2.826904e-04  0.000000   
1   0.012995    0.006002 -9.258189e-03  0.000000   
2   0.013054    0.004386 -2.060961e-03  0.000000   
3   0.015846    0.005415 -4.953691e-03  0.000000   
4   0.014464    0.006083 -5.354791e-03  0.000000   
0  20.182505    0.022784  7.634018e-01  0.970114   
1  19.302034    0.015000  7.388644e-01  0.972101   
2  16.817164    0.013104  7.640520e-01  0.969247   
3  17.046438    0.027631  6.412862e-01  0.971853   
4  17.098874    0.016161  6.798462e-01  0.971292   
0   0.230901    0.002000  4.275390e-01  1.000000   
1   0.245543    0.003196  3.799461e-01  1.000000   
2   0.239710    0.002996  5.980676e-01  1.000000   
3   0.233975

     fit_time  score_time       test_r2  train_r2  \
0    0.542248    0.007200 -4.460960e+26  0.348171   
1    0.553908    0.008503 -3.168009e+26  0.294230   
2    0.577683    0.007506 -4.148970e+26  0.202043   
3    0.582094    0.008337 -3.637989e+26  0.312907   
4    0.605664    0.009519 -1.412987e+26  0.160952   
0    0.199116    0.007647 -2.826904e-04  0.000000   
1    0.179464    0.008042 -9.258189e-03  0.000000   
2    0.180902    0.008194 -2.060961e-03  0.000000   
3    0.181499    0.007461 -4.953691e-03  0.000000   
4    0.174724    0.007593 -5.354791e-03  0.000000   
0  113.262835    0.029236  7.767850e-01  0.970929   
1  115.893982    0.030408  7.436310e-01  0.972769   
2  126.471838    0.042319  7.630551e-01  0.972358   
3  128.169093    0.041780  6.839547e-01  0.971621   
4  136.003128    0.031000  7.160761e-01  0.973293   
0    2.116168    0.010098  7.080007e-01  1.000000   
1    2.116663    0.009200  5.108056e-01  1.000000   
2    2.307041    0.011015  4.718378e-01  1.000

     fit_time  score_time       test_r2    train_r2  \
0   14.483877    0.057071 -3.720192e+26 -207.355682   
1   15.304017    0.054332 -2.238388e+26  -33.959904   
2   15.398905    0.062654 -1.590077e+26  -12.405779   
3   17.501071    0.062492 -5.407306e+26  -55.530974   
4   17.578229    0.074786 -3.245700e+26  -56.352379   
0    2.511738    0.069574 -2.826904e-04    0.000000   
1    2.626196    0.067916 -9.258189e-03    0.000000   
2    2.548746    0.071552 -2.060961e-03    0.000000   
3    2.673333    0.093200 -4.953691e-03    0.000000   
4    2.512774    0.058734 -5.354791e-03    0.000000   
0  755.960858    0.162299  7.841110e-01    0.968349   
1  767.778237    0.142791  6.931715e-01    0.970605   
2  807.515106    0.107248  7.282849e-01    0.968730   
3  936.902303    0.141052  6.361022e-01    0.969715   
4  975.898328    0.116618  7.062009e-01    0.971084   
0   18.269727    0.129847  6.246072e-01    1.000000   
1   16.073730    0.082081  5.152580e-01    1.000000   
2   15.293

     fit_time  score_time       test_r2  train_r2  \
0    0.140461    0.004850 -8.886461e+19  0.450064   
1    0.101104    0.003349 -2.600685e+22  0.398798   
2    0.104675    0.003516 -5.689491e+20  0.385057   
3    0.118141    0.003025 -2.626664e+21  0.405747   
4    0.114521    0.004000 -1.177143e+22  0.406137   
0    0.025457    0.004001 -2.826904e-04  0.000000   
1    0.021666    0.003199 -9.258189e-03  0.000000   
2    0.018713    0.004001 -2.060961e-03  0.000000   
3    0.018535    0.004013 -4.953691e-03  0.000000   
4    0.021326    0.004001 -5.354791e-03  0.000000   
0  110.230279    0.022285  7.437213e-01  0.968030   
1  107.457981    0.019215  7.206649e-01  0.969424   
2  109.038175    0.016873  7.505219e-01  0.965430   
3  109.364686    0.027830  6.346688e-01  0.968491   
4  113.621444    0.022594  6.576002e-01  0.970213   
0    1.443955    0.005046  5.706002e-01  1.000000   
1    1.496436    0.004999  5.799646e-01  1.000000   
2    1.433295    0.003106  5.675798e-01  1.000

    fit_time  score_time       test_r2  train_r2  \
0   0.073950    0.002001 -1.751999e+22  0.438661   
1   0.075439    0.003080 -4.230871e+22  0.409880   
2   0.067839    0.003011 -4.535775e+19  0.394996   
3   0.073899    0.003007 -4.398671e+21  0.392783   
4   0.071543    0.001521 -2.530856e+23  0.394152   
0   0.011399    0.003007 -2.826904e-04  0.000000   
1   0.011878    0.001516 -9.258189e-03  0.000000   
2   0.011719    0.003169 -2.060961e-03  0.000000   
3   0.014765    0.003527 -4.953691e-03  0.000000   
4   0.014374    0.001997 -5.354791e-03  0.000000   
0  84.769334    0.010784  7.325535e-01  0.968053   
1  86.853072    0.014663  7.116866e-01  0.969147   
2  88.024228    0.010728  7.418264e-01  0.966575   
3  89.501173    0.010770  6.540767e-01  0.968521   
4  91.008409    0.016015  6.602451e-01  0.969440   
0   1.225832    0.003003  4.748134e-01  1.000000   
1   1.247506    0.003642  5.573080e-01  1.000000   
2   1.210597    0.002879  4.442374e-01  1.000000   
3   1.290309

     fit_time  score_time       test_r2  train_r2  \
0    0.134385    0.002534 -1.695089e+10  0.546454   
1    0.091542    0.002167 -5.814253e+10  0.544490   
2    0.092483    0.002158 -4.600016e+21  0.524443   
3    0.093395    0.002693 -2.879803e+10  0.529081   
4    0.087826    0.002001 -1.925312e+10  0.541062   
0    0.023142    0.001999 -2.826904e-04  0.000000   
1    0.018947    0.003011 -9.258189e-03  0.000000   
2    0.019237    0.002186 -2.060961e-03  0.000000   
3    0.018564    0.002996 -4.953691e-03  0.000000   
4    0.019889    0.002997 -5.354791e-03  0.000000   
0  132.627546    0.020312  7.667155e-01  0.970251   
1  124.764918    0.010776  6.945127e-01  0.972016   
2  123.846298    0.011463  7.684496e-01  0.967898   
3  120.984548    0.010833  6.387108e-01  0.971243   
4  123.214187    0.011280  6.584294e-01  0.973795   
0    1.559931    0.002998  5.803455e-01  1.000000   
1    1.495930    0.002754  4.363787e-01  1.000000   
2    1.744861    0.002604  5.394879e-01  1.000

In [9]:
result=pd.DataFrame()
for item in range(8):
    result=result.append(summary[item])
result.to_excel(date + 'Predicting Temeprature Profiles' + enzyme +'5 CV.xlsx')

C:\Users\memre\AppData\Local\Temp\ipykernel_21364\3381766106.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_21364\3381766106.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_21364\3381766106.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_21364\3381766106.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_2136

In [10]:
df_res=result

In [11]:
df_res['test_root_mean_squared_error']=df_res['test_neg_root_mean_squared_error'].abs()
df_res['test_mean_absolute_error']=df_res['test_neg_mean_absolute_error'].abs()

df_res['train_root_mean_squared_error']=df_res['train_neg_root_mean_squared_error'].abs()
df_res['train_mean_absolute_error']=df_res['train_neg_mean_absolute_error'].abs()

In [12]:
test_res=df_res.groupby(['Algorithm', 'Sequence Representation Method'], as_index=False).agg({'test_r2':['mean','std'],
                                                                                             'test_root_mean_squared_error':['mean','std'],
                                                                                             'test_mean_absolute_error':['mean','std'],
                                                                                             'train_r2':['mean','std'],
                                                                                             'train_root_mean_squared_error':['mean','std'],
                                                                                             'train_mean_absolute_error':['mean','std']})

In [13]:
test_res.to_excel(date + 'Predicting Temperature Profiles' + enzyme +'5 CV mean and std.xlsx')